In [94]:
import mlflow
import mlflow.pyfunc
import pandas as pd
from sklearn.metrics import log_loss, f1_score, accuracy_score
import numpy as np

mlflow.set_tracking_uri("http://127.0.0.1:5500")

In [95]:
# model_name = "log_reg_model"
# model_version = 2
# model_uri = "models:/log_reg_model/2"
model = mlflow.sklearn.load_model("models:/log_reg_model/2")
print(model)

LogisticRegression(max_iter=1000, random_state=42)


In [96]:
df_prod = catalog.load('download_data_prod')
df_prod

[03/27/25 16:56:49] INFO     Loading data from download_data_prod (ParquetDataset)...           ]8;id=9248;file:///opt/miniconda3/envs/kedro-infnet-env/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=93616;file:///opt/miniconda3/envs/kedro-infnet-env/lib/python3.10/site-packages/kedro/io/data_catalog.py#401\401]8;;\

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
10,Jump Shot,Jump Shot,309,20000012,33.8063,-94,238,-118.3638,1,3,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,11
17,Jump Shot,Jump Shot,138,20000019,33.8183,-117,226,-118.3868,8,2,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,18
27,Jump Shot,Jump Shot,369,20000019,33.8583,-183,186,-118.4528,0,3,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,28
39,Jump Shot,Jump Shot,202,20000047,33.7723,-27,272,-118.2968,0,2,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,40
54,Jump Shot,Jump Shot,60,20000049,33.7973,-141,247,-118.4108,4,1,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-05,LAL vs. LAC,LAC,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30675,Jump Shot,Jump Shot,84,49900088,33.8283,-134,216,-118.4038,2,1,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30676
30681,Jump Shot,Jump Shot,218,49900088,33.7833,-18,261,-118.2878,0,2,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30682
30685,Jump Shot,Jump Shot,249,49900088,33.7943,81,250,-118.1888,7,3,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30686
30689,Jump Shot,Jump Shot,326,49900088,33.3653,-12,679,-118.2818,0,3,...,3PT Field Goal,Back Court(BC),Backcourt,Back Court Shot,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30690


In [97]:
features = df_prod[['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']]
df_prod_clean = features.dropna(subset=['shot_made_flag'])
df_prod_clean

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
10,33.8063,-118.3638,1,3,0,25,0.0
17,33.8183,-118.3868,8,2,0,25,1.0
27,33.8583,-118.4528,0,3,0,26,0.0
39,33.7723,-118.2968,0,2,0,27,1.0
55,33.8503,-118.4248,2,1,0,24,0.0
...,...,...,...,...,...,...,...
30675,33.8283,-118.4038,2,1,1,25,0.0
30681,33.7833,-118.2878,0,2,1,26,1.0
30685,33.7943,-118.1888,7,3,1,26,0.0
30689,33.3653,-118.2818,0,3,1,67,0.0


In [98]:
if len(df_prod_clean) < len(features):
        print(f"Removidas {len(features) - len(df_prod_clean)} linhas com NaN em shot_made_flag")

Removidas 1014 linhas com NaN em shot_made_flag


In [99]:
X = df_prod_clean[["lat", "lon", "minutes_remaining", "period", "playoffs", "shot_distance"]]
y_true = df_prod_clean["shot_made_flag"]

from sklearn.preprocessing import RobustScaler

# Aplicar RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)


In [105]:
y_pred = model.predict(X_scaled)
y_pred_proba = model.predict_proba(X_scaled)[:, 1]

In [110]:
# Calcular as métricas de avaliação
f1 = f1_score(y_true, y_pred)
logloss = log_loss(y_true, y_pred_proba)
accuracy = accuracy_score(y_true, y_pred)

print(f"F1-score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Log Loss: {logloss:.4f}")

F1-score: 0.3499
Accuracy: 0.5798
Log Loss: 0.6663
